In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-154540
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-154540


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
aml_name = "udacity-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=4)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

Creating new AML compute context.
InProgress.....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [15]:
#!pip install 'azureml-sdk[notebooks]'

In [21]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
      '--C': uniform( 0.2, 5),
      '--max_iter': choice(100, 150, 200, 250, 300, 400)
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 2, delay_evaluation = 5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './', 
                entry_script = 'train.py', 
                compute_target = aml_name,
                pip_packages = ["pandas"]
                )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                     hyperparameter_sampling = ps,
                                     policy = policy,
                                     primary_metric_name = "Accuracy",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 40,
                                     max_concurrent_runs = 4)

In [22]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_80261dd5-c4d5-4d19-8584-80cd897b916f
Web View: https://ml.azure.com/runs/HD_80261dd5-c4d5-4d19-8584-80cd897b916f?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-154540/workspaces/quick-starts-ws-154540&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-15T00:34:39.423751][API][INFO]Experiment created<END>\n""<START>[2021-08-15T00:34:40.271250][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-08-15T00:34:40.457405][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_80261dd5-c4d5-4d19-8584-80cd897b916f
Web View: https://ml.azure.com/runs/HD_80261dd5-c4d5-4d19-8584-80cd897b916f?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-154540/workspaces/quick-starts-ws-154540&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_80261dd5-c4d5-4d19-8584-80cd897b916f',
 'target': 'udacity-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-15T00:34:39.193073Z',
 'endTimeUtc': '2021-08-15T00:58:49.012486Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '089cc6f0-e083-42b7-8628-badd3f2d5499',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.9150227617602428',
  'best_child_run_id': 'HD_80261dd5-c4d5-4d19-8584-80cd897b916f_2',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg154540.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_80261dd5-c4d5-4d19-8584-80cd897b916f/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr

In [24]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Parameter Values:', parameter_values)

Best Run Id:  HD_80261dd5-c4d5-4d19-8584-80cd897b916f_2
Accuracy: 0.9150227617602428
Parameter Values: ['--C', '2.621997802065188', '--max_iter', '300']


In [25]:
joblib.dump(value=best_run.get_details()['runDefinition']['arguments'], 
            filename=os.path.join('outputs', 'best_hyperdrive_model.joblib'))

['outputs/best_hyperdrive_model.joblib']

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url)
pd_data = ds.to_pandas_dataframe()
display(pd_data.head())

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [9]:
from azureml.core import Workspace, Dataset
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['target'] = y
local_path = 'DT.csv'
x.to_csv(local_path)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='.', target_path='data')

# create a dataset referencing the cloud location
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/DT.csv'))])


Uploading an estimated of 18 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 18
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 18
Uploading ./automl.log
Uploaded ./automl.log, 3 files out of an estimated total of 18
Uploading ./test.ipynb
Uploaded ./test.ipynb, 4 files out of an estimated total of 18
Uploading ./test.ipynb.amltmp
Uploaded ./test.ipynb.amltmp, 5 files out of an estimated total of 18
Uploading ./train.py
Uploaded ./train.py, 6 files out of an estimated total of 18
Uploading ./train.py.amltmp
Uploaded ./train.py.amltmp, 7 files out of an estimated total of 18
Uploading ./udacity-project.ipynb
Uploaded ./udacity-project.ipynb, 8 files out of an estimated total of 18
Uploading ./udacity-project.ipynb.amltmp
Uploaded ./udacity-project.ipynb.amltmp, 9 files out of an estimated total of 18
Uploading ./.ipynb_aml_checkpoints/udacity-project-checkpoint2021-7-15-1-59-34Z.ipynb
Uploaded ./.i

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='target',
    n_cross_validations=5)


In [5]:
from azureml.core import VERSION
print("Azure ML SDK Version: ", VERSION)

Azure ML SDK Version:  1.32.0


In [9]:
pip install -r /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Submit your automl run
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment

expirement = Experiment(workspace=ws, name="automl_remote")  
auto_run = expirement.submit(automl_config, show_output=True)
RunDetails(auto_run).show()

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_remote,AutoML_f1de0178-f5fd-406f-925e-52075b5e7b42,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = auto_run.get_output()
best_run.register_model('automl_model.pkl', model_path = './outputs/')